In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

# Using PiNN as a Neural Network Potential

Here the model is trained on the [ANI-1](https://figshare.com/articles/ANI-1_data_set_20M_DFT_energies_for_non-equilibrium_small_molecules/5287732) (20M DFT energies for non-equilibrium small molecules) dataset.

In [71]:
from pinn.models import potential_model
from pinn.calculator import PiNN_calc

params = {'model_dir': '../../model_dir/PiNN_ANI',
          'network': 'pinn_network',
          'netparam': {'atom_types':[1, 6, 7, 8]},
          'train': {}}

model = potential_model(params)
calc = PiNN_calc(model, unit=units.kcal/units.mol)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../../model_dir/PiNN_ANI', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f827c12da58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [102]:
from ase.calculators.tip3p import TIP3P, rOH, angleHOH
atoms = g2['CH4']

vol = ((18.01528 / 6.022140857e23) / (0.9982 / 1e24))**(1 / 3.)
atoms.set_cell((vol, vol, vol))
atoms.center()

atoms = atoms.repeat((3, 3, 3))
atoms.set_pbc(True)
atoms.set_calculator(calc)

## Run some molecular dynamics

In [107]:
from ase import units
from ase.md.verlet import VelocityVerlet

dyn = VelocityVerlet(atoms, dt=0.5 * units.fs,
                     trajectory='md.traj', logfile='md.log')
dyn.run(1000)

In [108]:
import nglview as nv
from ase.io import read
nv.show_asetraj(read('md.traj', index=':'))

NGLWidget(count=1000)

## Run some NEB 

In [83]:
from ase import Atoms
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms

slab = fcc111('C',size=(4,4,2), a=2)
slab.set_pbc((0,0,0))

# Initial state.
# Add the N2 molecule oriented at 60 degrees:
d = 1.10 # N2 bond length
N2mol = Atoms('N2',positions=[[0.0,0.0,0.0],[0.5*3**0.5*d,0.5*d,0.0]])
add_adsorbate(slab,N2mol,height=1.5,position='fcc')

# Use the EMT calculator for the forces and energies:
slab.set_calculator(calc)

# We don't want to worry about the Cu degrees of freedom,
# so fix these atoms:

mask = [atom.symbol == 'C' for atom in slab]
slab.set_constraint(FixAtoms(mask=mask))
nv.show_ase(slab)

NGLWidget()

In [84]:
slab.get_potential_energy()

46.71679705280212

In [85]:
from ase.optimize import QuasiNewton
from ase.io import write

relax = QuasiNewton(slab)
relax.run(fmax=0.05)
print('initial state:', slab.get_potential_energy())
write('N2.traj', slab)

# Now the final state.
# Move the second N atom to a neighboring hollow site:
slab[-1].position[0] = slab[-2].position[0] + 0.25 * slab.cell[0,0]
slab[-1].position[1] = slab[-2].position[1]
# and relax.
relax.run()
print('final state:  ', slab.get_potential_energy())
write('2N.traj', slab)

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 14:07:36       46.716797        2.0666
BFGSLineSearch:    1[  2] 14:07:36       46.640735        0.6030
BFGSLineSearch:    2[  9] 14:07:37       43.252592        3.4705
BFGSLineSearch:    3[ 10] 14:07:37       43.135828        1.3636
BFGSLineSearch:    4[ 12] 14:07:37       43.127645        0.2628
BFGSLineSearch:    5[ 13] 14:07:37       43.114596        0.5680
BFGSLineSearch:    6[ 15] 14:07:37       43.077547        0.7577
BFGSLineSearch:    7[ 17] 14:07:37       43.022757        0.3946
BFGSLineSearch:    8[ 18] 14:07:37       43.002412        0.1807
BFGSLineSearch:    9[ 19] 14:07:38       43.000315        0.2078
BFGSLineSearch:   10[ 20] 14:07:38       42.999437        0.0937
BFGSLineSearch:   11[ 21] 14:07:38       42.998907        0.0395
initial state: 42.99891271846455
BFGSLineSearch:   11[ 21] 14:07:38       46.487876        8.9827
BFGSLineSearch:   12[ 23] 14:07:38       44.528942       

In [89]:
cd /home/yunqi/work/pinn_proj/PiNN_lab/lab_session/Day\ 2

/home/yunqi/work/pinn_proj/PiNN_lab/lab_session/Day 2


In [91]:
nv.show_ase(read('2N.traj'))


NGLWidget()

In [52]:
import numpy as np

from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
from ase.neb import SingleCalculatorNEB as NEB
from ase.optimize.fire import FIRE as QuasiNewton
from ase.io import read

# Read the previous configurations
initial = read('N2.traj')
final = read('2N.traj')

initial.set_calculator(calc)
final.set_calculator(calc)

#  Make 9 images (note the use of copy)
configs = [initial.copy() for i in range(8)] + [final]

# As before, fix the Cu atoms
constraint = FixAtoms(mask=[atom.symbol != 'N' for atom in initial])
for config in configs:
    config.set_calculator(calc)
    config.set_constraint(constraint)

# Make the NEB object, interpolate to guess the intermediate steps
band = NEB(configs)
band.interpolate()

relax = QuasiNewton(band)

# Do the calculation
relax.run()

# Compare intermediate steps to initial energy
e0 = initial.get_potential_energy()
for config in configs:
    d = config[-2].position - config[-1].position
    print(np.linalg.norm(d), config.get_potential_energy() - e0)

      Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
FIRE:    0 13:34:53       -2.222201*       0.0371
2.5104382021613705 1.6542092851068446e-07
2.5253504068946366 0.0010992220703864852
2.5446412424923093 0.002514728956210188
2.5682120420714667 0.004249167392328168
2.5959462239705937 0.006303199062454912
2.627711964259339 0.008671199655018391
2.6633650146261143 0.011346552332875959
2.7027515423758897 0.014319497261241487
2.7457108848446397 0.017576966186757748
